TT. News Collection for Sentiment Analysis

In [1]:
# Librerías.
import requests
import csv
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import datetime

URLs of RSS sites.

In [2]:
# URL de páginas relacionadas con noticias de criptomonedas.

# HTTP - 200 OK.
""" RSS Feeds (XML). """
urls_dict = {
    #'coindesk': 'https://www.coindesk.com/arc/outboundfeeds/rss',
    'cointelegraph': 'https://cointelegraph.com/rss/tag/bitcoin',
    'bitcoinmagazine': 'https://bitcoinmagazine.com/.rss/full/',
    'cryptopotato': 'https://cryptopotato.com/feed/',
    'cryptoslate': 'https://cryptoslate.com/feed/',
    'cryptonews': 'https://cryptonews.com/news/feed/'
}

In [3]:
# Ruta del archivo CSV para almacenar las noticias
csv_file = 'news_data.csv'

# Cargar datos previos si el archivo existe
if os.path.exists(csv_file):
    existing_df = pd.read_csv(csv_file)
else:
    existing_df = pd.DataFrame()  # Crear un DataFrame vacío si no existe el archivo

Getting soup and items for each site.

In [4]:
# Crear una lista para almacenar características de las noticias.
news_data = []

# Hacer solicitudes HTTP a las páginas web.
for name, url in urls_dict.items():
    response = requests.get(url)
    print(f'Response from {name}:')
    print(response)

    if response.status_code == 200:
        # Parsear el contenido de las respuestas con BeautifulSoup.
        soup = BeautifulSoup(response.content, 'xml')

        # Extraer todos los elementos <item>.
        items = soup.find_all('item')
        print(f'Items from {name}:')
        
        # ----------------------------------------------------------------------- CoinDesk News. ----------------------------------------------------------------------- #
        #"""
        if name == 'coindesk':
            # Iterar cada elemento <item> y extraer características de las noticias de CoinDesk
            for item in items:
                title = item.find('title').text
                pub_date = item.find('pubDate').text
                # Convertir pubDate a formato "EEE, dd, MMM, yyyy"
                pub_date = datetime.strptime(pub_date, '%a, %d %b %Y %H:%M:%S %z').strftime('%a, %d %b %Y')
                link = item.find('link').text
                description = item.find('description').text if item.find('description') else 'No description available'
                
                # Filtrar solo las noticias de Bitcoin
                if 'bitcoin' in title.lower() or 'BTC' in title.lower():
                    # Hacer una solicitud HTTP a la URL de la noticia para obtener el contenido completo.
                    article_response = requests.get(link)
                    article_soup = BeautifulSoup(article_response.content, 'html.parser')
                
                    # Extraer el contenido de cada noticia.
                    article_paragraphs = article_soup.find_all('p')
                    article_content = ' '.join([paragraph.text for paragraph in article_paragraphs])
                    
                    # Añadir los datos de la noticia a la lista.
                    news_data.append({
                        'site': 'CoinDesk',
                        'title': title,
                        'pubDate': pub_date,
                        'link': link,
                        'description': description,
                        'content': article_content
                    })
            #"""
        # ----------------------------------------------------------------------- CoinTelegraph News. ----------------------------------------------------------------------- #
        if name == 'cointelegraph':
        # Iterar cada elemento <item> y las características de cada noticia. 
            for item in items:
                title = item.find('title').text
                pub_date = item.find('pubDate').text
                # Convertir pubDate a formato "EEE, dd, MMM, yyyy"
                pub_date = datetime.strptime(pub_date, '%a, %d %b %Y %H:%M:%S %z').strftime('%a, %d %b %Y')
                link = item.find('link').text
                #description = item.find('description').text if item.find('description') else 'No description available'
                #news_data.append({'title': title, 'pubDate': pub_date, 'link': link, 'description': description})
            
                description_html = item.find('description').text if item.find('description') else 'No description available'
                
                # Parsear el contenido HTML de la descripción y extraer solo el texto.
                description_soup = BeautifulSoup(description_html, 'html.parser')
                description = description_soup.get_text(strip=True)
            
                # Filtrar solo las noticias de Bitcoin
                if 'bitcoin' in title.lower() or 'BTC' in title.lower():
                    # Hacer una solicitud HTTP a la URL de la noticia para obtener el contenido completo de la noticia en cuestión.
                    #article_response = requests.get(link)
                    #article_soup = BeautifulSoup(article_response.content, 'html.parser')
                    
                    # Extraer el contenido de cada noticia.
                    #article_paragraphs = article_soup.find_all('p')
                    #article_content = ' '.join([paragraph.text for paragraph in article_paragraphs])
            
                    # Añadir los datos de la noticia a la lista.
                    news_data.append({
                        'site': 'CoinTelegraph',
                        'title': title,
                        'pubDate': pub_date,
                        'link': link,
                        'description': description
                        #'content': article_content
                    })
        
        # ----------------------------------------------------------------------- BitcoinMagazine News. ----------------------------------------------------------------------- #
        if name == 'bitcoinmagazine':
        # Iterar cada elemento <item> y las características de cada noticia.
            for item in items:
                title = item.find('title').text
                pub_date = item.find('pubDate').text

                # Eliminar el sufijo 'GMT' y convertir la fecha al formato deseado
                pub_date_cleaned = pub_date.replace(' GMT', '')  # Eliminar ' GMT'
                pub_date_parsed = datetime.strptime(pub_date_cleaned, '%a, %d %b %Y %H:%M:%S') 
                link = item.find('link').text
                description = item.find('description').text if item.find('description') else 'No description available'
                content = item.find('content:encoded').text if item.find('content:encoded') else 'No description available'
                #news_data.append({'title': title, 'pubDate': pub_date, 'link': link, 'description': description})
            
            
                cleaned_content = BeautifulSoup(content, 'html.parser').get_text()
                
                # Hacer una solicitud HTTP a la URL de la noticia para obtener el contenido completo de la noticia en cuestión.
                #article_response = requests.get(link)
                #article_soup = BeautifulSoup(article_response.content, 'html.parser')
                
                # Extraer el contenido de cada noticia.
                #article_paragraphs = content.find_all('p')
                #article_content = ' '.join([paragraph.text for paragraph in article_paragraphs])
            
                # Añadir los datos de la noticia a la lista.
                news_data.append({
                    'site': 'Bitcoin Magazine',
                    'title': title,
                    'pubDate': pub_date,
                    'link': link,
                    'description': description,
                    'content': cleaned_content
                })

        # ----------------------------------------------------------------------- CryptoNews News. ----------------------------------------------------------------------- #
        if name == 'cryptonews':
        # Iterar cada elemento <item> y las características de cada noticia.
            for item in items:
                title = item.find('title').text
                pub_date = item.find('pubDate').text
                # Convertir pubDate a formato "EEE, dd, MMM, yyyy"
                pub_date = datetime.strptime(pub_date, '%a, %d %b %Y %H:%M:%S %z').strftime('%a, %d %b %Y')
                link = item.find('link').text
                description_html = item.find('description').text if item.find('description') else 'No description available'
                content = item.find('content:encoded').text if item.find('content:encoded') else 'No description available'
                #news_data.append({'title': title, 'pubDate': pub_date, 'link': link, 'description': description})
                cleaned_content = BeautifulSoup(content, 'html.parser').get_text()
            
                # Parsear el contenido HTML de la descripción y extraer solo el texto.
                description_soup = BeautifulSoup(description_html, 'html.parser')
                description = description_soup.get_text(strip=True)
                
                # Filtrar solo las noticias de Bitcoin.
                if 'bitcoin' in title.lower() or 'BTC' in title.lower():
                    # Hacer una solicitud HTTP a la URL de la noticia para obtener el contenido completo de la noticia en cuestión.
                    #article_response = requests.get(link)
                    #article_soup = BeautifulSoup(article_response.content, 'html.parser')
                
                    # Extraer el contenido de cada noticia.
                    #article_paragraphs = content.find_all('p')
                    #article_content = ' '.join([paragraph.text for paragraph in article_paragraphs])
            
                    # Añadir los datos de la noticia a la lista.
                    news_data.append({
                        'site': 'Crypto News',
                        'title': title,
                        'pubDate': pub_date,
                        'link': link,
                        'description': description,
                        'content': cleaned_content
                    })

        # ----------------------------------------------------------------------- CryptoPotato News. ----------------------------------------------------------------------- #
        if name == 'cryptopotato':
        # Iterar cada elemento <item> y las características de cada noticia.
            for item in items:
                title = item.find('title').text
                pub_date = item.find('pubDate').text
                # Convertir pubDate a formato "EEE, dd, MMM, yyyy"
                pub_date = datetime.strptime(pub_date, '%a, %d %b %Y %H:%M:%S %z').strftime('%a, %d %b %Y')
                link = item.find('link').text
                description = item.find('description').text if item.find('description') else 'No description available'
                content = item.find('content:encoded').text if item.find('content:encoded') else 'No description available'
                #news_data.append({'title': title, 'pubDate': pub_date, 'link': link, 'description': description})
                cleaned_content = BeautifulSoup(content, 'html.parser').get_text()
            
                # Filtrar solo las noticias de Bitcoin.
                if 'bitcoin' in title.lower() or 'BTC' in title.lower():
                    # Hacer una solicitud HTTP a la URL de la noticia para obtener el contenido completo de la noticia en cuestión.
                    #article_response = requests.get(link)
                    #article_soup = BeautifulSoup(article_response.content, 'html.parser')
                
                    # Extraer el contenido de cada noticia.
                    #article_paragraphs = content.find_all('p')
                    #article_content = ' '.join([paragraph.text for paragraph in article_paragraphs])
            
                    # Añadir los datos de la noticia a la lista.
                    news_data.append({
                        'site': 'Crypto Potato',
                        'title': title,
                        'pubDate': pub_date,
                        'link': link,
                        'description': description,
                        'content': cleaned_content
                    })
                    
        # ----------------------------------------------------------------------- CryptoSlate News. ----------------------------------------------------------------------- #
        if name == 'cryptoslate':
        # Iterar cada elemento <item> y las características de cada noticia.
            for item in items:
                title = item.find('title').text
                pub_date = item.find('pubDate').text
                # Convertir pubDate a formato "EEE, dd, MMM, yyyy"
                pub_date = datetime.strptime(pub_date, '%a, %d %b %Y %H:%M:%S %z').strftime('%a, %d %b %Y')
                link = item.find('link').text
                description_html = item.find('description').text if item.find('description') else 'No description available'
                content = item.find('content:encoded').text if item.find('content:encoded') else 'No description available'
                #news_data.append({'title': title, 'pubDate': pub_date, 'link': link, 'description': description})
                cleaned_content = BeautifulSoup(content, 'html.parser').get_text()
            
                # Parsear el contenido HTML de la descripción y extraer solo el texto.
                description_soup = BeautifulSoup(description_html, 'html.parser')
                description = description_soup.get_text(strip=True)
                
                # Filtrar solo las noticias de Bitcoin.
                if 'bitcoin' in title.lower() or 'BTC' in title.lower():
                    # Hacer una solicitud HTTP a la URL de la noticia para obtener el contenido completo de la noticia en cuestión.
                    #article_response = requests.get(link)
                    #article_soup = BeautifulSoup(article_response.content, 'html.parser')
                
                    # Extraer el contenido de cada noticia.
                    #article_paragraphs = content.find_all('p')
                    #article_content = ' '.join([paragraph.text for paragraph in article_paragraphs])
            
                    # Añadir los datos de la noticia a la lista.
                    news_data.append({
                        'site': 'Crypto Slate',
                        'title': title,
                        'pubDate': pub_date,
                        'link': link,
                        'description': description,
                        'content': cleaned_content
                    })
        
        print("\n--------------------------------------------------------------------------------------------------------\n")
    else:
        print(f'Failed to retrieve data from {name}. Status code: {response.status_code}')

Response from cointelegraph:
<Response [200]>
Items from cointelegraph:

--------------------------------------------------------------------------------------------------------

Response from bitcoinmagazine:
<Response [200]>
Items from bitcoinmagazine:

--------------------------------------------------------------------------------------------------------

Response from cryptopotato:
<Response [403]>
Failed to retrieve data from cryptopotato. Status code: 403
Response from cryptoslate:
<Response [200]>
Items from cryptoslate:

--------------------------------------------------------------------------------------------------------

Response from cryptonews:
<Response [200]>
Items from cryptonews:

--------------------------------------------------------------------------------------------------------



Storing all the BTC news in a DataFrame.

In [5]:
# Convertir la lista de diccionarios a un DataFrame de pandas.
new_df = pd.DataFrame(news_data)

# Concatenar el nuevo DataFrame con los datos existentes.
final_df = pd.concat([existing_df, new_df], ignore_index=True)

# Eliminar duplicados basados en el título.
final_df.drop_duplicates(subset=['title'], inplace=True)

# Limpieza de valores NaN en columna 'content'.
final_df['content'] = final_df['content'].fillna('')

# Limpieza del caracter salto de línea '\n' en la columna 'content'.
final_df['content'] = final_df['content'].str.replace('\n', ' ')

# Guardar el DataFrame actualizado en el archivo CSV.
final_df.to_csv(csv_file, index=False)

# Mostrar las noticias acumuladas.
print(final_df)

               site                                              title  \
0     CoinTelegraph   Bitcoin in ‘seasonal slog’ with slim chance o...   
1     CoinTelegraph   Bitcoin traders celebrate ‘good news’ as BTC ...   
2     CoinTelegraph   Bitcoin price shows strength above its recent...   
3     CoinTelegraph   Bitcoin ETFs record combined $1.2B in outflow...   
4     CoinTelegraph   Is crypto entering a bear market? — 5 Things ...   
...             ...                                                ...   
1058   Crypto Slate  MicroStrategy now controls over 2% of Bitcoin ...   
1059    Crypto News  Riot Plans $500M Convertible Notes Offering to...   
1060    Crypto News  Bitcoin’s Rally Could Hit Pause in 2025, Says ...   
1061    Crypto News  El Salvador to Adjust Bitcoin Policy for $1.3 ...   
1062    Crypto News  Schiff Brands Bitcoin ‘Public Enemy #1’ Amid B...   

               pubDate                                               link  \
0     Tue, 10 Sep 2024  https://co